In [1]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import balanced_accuracy_score

tf.enable_eager_execution()

#tf.executing_eagerly()

D:\Anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1ty

### Data inspection

In [2]:
df = pd.read_csv("data/train.csv")
df.head()
df.isnull().sum()
df.notnull().sum()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
print("Different values at the 'Embarked' column: %d" %df['Embarked'].value_counts().shape[0])
print("Different values at the 'Cabin' column: %d" %df['Cabin'].value_counts().shape[0])
print("Different values at the 'Ticket' column: %d" %df['Ticket'].value_counts().shape[0])
print("Different values at the 'Name' column: %d" %df['Name'].value_counts().shape[0])
print("Differnt values at the 'Sex' column: %d" %df['Sex'].value_counts().shape[0])


Different values at the 'Embarked' column: 3
Different values at the 'Cabin' column: 147
Different values at the 'Ticket' column: 681
Different values at the 'Name' column: 891
Differnt values at the 'Sex' column: 2


### Data preprocessing

Replacing null values  
Removing columns with many missing values and unuseful like 'Name' and 'Ticket'  
One-hot encoding categorical variables for Neural Network

In [4]:
# Replace null Values (np.nan) with mean
df['Age'] = df['Age'].replace(np.nan, df['Age'].mean())
# Replacing the null values with the most frequent value
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].value_counts().index[0])
# Replacing null values with Unknown Class
df['Cabin'] = df['Cabin'].fillna('Unknown')

# Removing unused columns
UNUSED_COLUMNS = ["Name", "Ticket", "Cabin"]
df = df.drop(UNUSED_COLUMNS, axis=1)

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['Embarked', 'Sex'])

### Function to split the dataset into train, validation and test

In [5]:
def get_dataset_partitions_pd(df, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1

    # Specify seed to always have the same split distribution between runs
    # df.samples shuffles the dataframe
    df_sample = df.sample(frac=1, random_state=12)
    indices_or_sections = [int(train_split * len(df)), int((1 - val_split) * len(df))]
    
    train_ds, test_ds, val_ds = np.split(df_sample, indices_or_sections)
    
    return train_ds, val_ds, test_ds

train_df, val_df, test_df = get_dataset_partitions_pd(df, train_split=0.8, val_split=0, test_split=0.2)
y_train = train_df['Survived']
X_train = train_df.drop(columns='Survived')
y_test = test_df['Survived']
X_test = test_df.drop(columns='Survived')

### Neural Network

In [6]:
# Softmax for multiclass classification problem
# Relu activation function
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(0.001)
loss = tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy()

model.compile(optimizer, loss=loss, metrics=[acc])

history = model.fit(X_train, y_train,
            batch_size=50,
            epochs=80,
            verbose=1,
            validation_split=0.2)

Train on 569 samples, validate on 143 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/80
569/569 [==============================] - 0s 357us/sample - loss: 8.1777 - binary_accuracy: 0.3972 - val_loss: 8.5871 - val_binary_accuracy: 0.3427
Epoch 2/80
569/569 [==============================] - 0s 21us/sample - loss: 6.5280 - binary_accuracy: 0.4077 - val_loss: 4.0063 - val_binary_accuracy: 0.6154
Epoch 3/80
569/569 [==============================] - 0s 19us/sample - loss: 4.4682 - binary_accuracy: 0.6239 - val_loss: 3.3102 - val_binary_accuracy: 0.5315
Epoch 4/80
569/569 [==============================] - 0s 19us/sample - loss: 3.0945 - binary_accuracy: 0.4815 - val_loss: 3.2068 - val_binary_accuracy: 0.6014
Epoch 5/80
569/569 [==============================] - 0s 21us/sample - loss: 2.8132 - binary_accuracy: 0.5325 - val_loss: 2.6796 - val_binary_accuracy: 0.5734
Epoch 6/80
569/569 [==============================] - 0s 19us/sa

569/569 [==============================] - 0s 19us/sample - loss: 0.5176 - binary_accuracy: 0.7504 - val_loss: 0.4925 - val_binary_accuracy: 0.7762
Epoch 50/80
569/569 [==============================] - 0s 19us/sample - loss: 0.5074 - binary_accuracy: 0.7540 - val_loss: 0.4898 - val_binary_accuracy: 0.7902
Epoch 51/80
569/569 [==============================] - 0s 19us/sample - loss: 0.4965 - binary_accuracy: 0.7663 - val_loss: 0.4809 - val_binary_accuracy: 0.8042
Epoch 52/80
569/569 [==============================] - 0s 18us/sample - loss: 0.4924 - binary_accuracy: 0.7557 - val_loss: 0.5283 - val_binary_accuracy: 0.7413
Epoch 53/80
569/569 [==============================] - 0s 18us/sample - loss: 0.5624 - binary_accuracy: 0.7276 - val_loss: 0.5993 - val_binary_accuracy: 0.6923
Epoch 54/80
569/569 [==============================] - 0s 18us/sample - loss: 0.6071 - binary_accuracy: 0.7153 - val_loss: 0.7737 - val_binary_accuracy: 0.5874
Epoch 55/80
569/569 [==============================]

In [9]:
y_pred = model.predict(X_test)
metric = tf.keras.metrics.BinaryAccuracy()
metric.update_state(y_test,y_pred)
nn_accuracy = metric.result().numpy()
print("Neural Networks Accuracy: %.2f%%" % (nn_accuracy * 100.0))

Neural Networks Accuracy: 81.56%


### Random Forests

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)
model.fit(X_train, y_train)
y_prediction = model.predict(X_test)

In [11]:
rf_accuracy = accuracy_score(y_test, y_prediction)
print("RF Accuracy: %.2f%%" % (rf_accuracy * 100.0))

RF Accuracy: 83.80%


### XGBoost

In [13]:
import xgboost
from xgboost.sklearn import XGBClassifier

In [14]:
xgb_model = xgboost.XGBClassifier(objective="binary:logistic", n_estimators=30, random_state=42, eval_metric=["auc", "error", "error@0.6"])
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
y_pred_xgb = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print("Xgb Accuracy: %.2f%%" % (xgb_accuracy * 100.0))

[0]	validation_0-auc:0.91038	validation_0-error:0.16201	validation_0-error@0.6:0.19553
[1]	validation_0-auc:0.90741	validation_0-error:0.17318	validation_0-error@0.6:0.15643
[2]	validation_0-auc:0.91257	validation_0-error:0.18436	validation_0-error@0.6:0.16760
[3]	validation_0-auc:0.91296	validation_0-error:0.17318	validation_0-error@0.6:0.16760
[4]	validation_0-auc:0.91322	validation_0-error:0.16760	validation_0-error@0.6:0.16201
[5]	validation_0-auc:0.90876	validation_0-error:0.16760	validation_0-error@0.6:0.16760
[6]	validation_0-auc:0.91141	validation_0-error:0.17318	validation_0-error@0.6:0.16760
[7]	validation_0-auc:0.90650	validation_0-error:0.18436	validation_0-error@0.6:0.16760
[8]	validation_0-auc:0.90534	validation_0-error:0.15643	validation_0-error@0.6:0.17318
[9]	validation_0-auc:0.90560	validation_0-error:0.16760	validation_0-error@0.6:0.17318
[10]	validation_0-auc:0.90624	validation_0-error:0.16760	validation_0-error@0.6:0.17318
[11]	validation_0-auc:0.90198	validation_0

D:\Anaconda\envs\tf\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
